In [1]:
!pip install vaderSentiment

In [2]:
!pip install nltk

In [3]:
!pip install stop_words

# Importing all libraries

In [4]:
import json
import pandas as pd
import numpy as np
import os
import re
import sys
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import f1_score, precision_score, recall_score
import collections
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import stop_words
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import gzip
import pickle
import tensorflow as tf

In [6]:
pre_order_df = pd.read_csv('Datasets/Inflight Service_Pre order data.csv')
inventory_df = pd.read_csv('Datasets/Inflight Service_Inventory data.csv')

In [7]:
pre_order_df = pre_order_df[['flight_number','meal_group', 'meal_category', 'meal_short_description','meal_description',  'pre_order_qty']]

In [8]:
totalpreord_per_meal = pre_order_df.groupby('meal_short_description')['pre_order_qty'].sum().reset_index()

In [12]:
totalpreord_per_meal = totalpreord_per_meal.sort_values(by='pre_order_qty', ascending=False)

In [13]:
totalpreord_per_meal.head(5)

,meal_short_description,pre_order_qty
102,Za’atar chicken,36135
12,Brioche cheeseburger,33687
75,Scrambled eggs,26099
45,Impossible™ Meatball Bowl,20757
8,Belgium waffle,18568


In [14]:
most_preordered = totalpreord_per_meal.sort_values(by='pre_order_qty', ascending=False).head(10)

# Sort the DataFrame by pre_order_qty to find least preordered meals
least_preordered = totalpreord_per_meal.sort_values(by='pre_order_qty').head(10)

print("10 Most Preordered Meals:")
print(most_preordered)

print("\n10 Least Preordered Meals:")
print(least_preordered)

10 Most Preordered Meals:
                                meal_short_description  pre_order_qty
102                                    Za’atar chicken          36135
12                                Brioche cheeseburger          33687
75                                      Scrambled eggs          26099
45                           Impossible™ Meatball Bowl          20757
8                                       Belgium waffle          18568
89                                             Takeoff          13437
90                                               Tapas          13008
93                           Tomato and kale baked egg          11235
101  Za'atar chicken with spinach, roasted peppers,...           8694
96                   Vegetarian chili and polenta cake           8063

10 Least Preordered Meals:
                               meal_short_description  pre_order_qty
43                              I will decide onboard              0
44               I would like to go di

In [15]:
inventory_df.head(10)

,flight_number,departure_station_code,arrival_station_code,scheduled_departure_dtl,entree_product_id,entree_description,entree_code,planned_entree_count,consumed_entree_count
0,2634,IAX,PDX,9/1/2022,58019,Belgium waffle,WF,20,32
1,2666,ORX,IAX,9/1/2022,60549,Vegetarian chili and polenta cake,V,3,0
2,2310,DEX,SAX,9/2/2022,58981,Scrambled eggs,O,9,7
3,1712,SFX,CLX,9/4/2022,58764,Mediterranean Falafel Bowl,V,1,0
4,1225,LAX,HNX,9/8/2022,60549,Vegetarian chili and polenta cake,V,1,0
5,2637,CLX,MCX,9/12/2022,58019,Belgium waffle,WF,16,16
6,1691,DEX,SAX,9/12/2022,58574,Belgium waffle,WF,6,2
7,592,SEX,EWX,9/13/2022,60983,Zatar chicken,C,22,22
8,1618,LAX,ORX,9/13/2022,60987,Impossible Meatball Bowl,V,7,4
9,2087,IAX,LAX,9/13/2022,60983,Zatar chicken,C,10,9


In [16]:
inventory_df = inventory_df[['flight_number', 'entree_product_id', 'entree_description', 'planned_entree_count', 
                             'consumed_entree_count' ]]

In [17]:
invent_newdf = inventory_df.groupby(['entree_description'])[['planned_entree_count', 'consumed_entree_count']].sum().reset_index()

In [19]:
invent_newdf.head(5)

,entree_description,planned_entree_count,consumed_entree_count
0,ASIAN VEGETARIAN MEAL,606,354
1,Arepa and black bean relish,2,0
2,Bangus fish,165,0
3,Beef and onion stew in red wine sauce,249,0
4,Beef burger,145,105


# Food Wastage

In [20]:
invent_newdf['food_wastage'] = invent_newdf['planned_entree_count'] - invent_newdf['consumed_entree_count']

# Filter rows with positive food wastage
food_wastage_df = invent_newdf[invent_newdf['food_wastage'] > 0]

# Sort by food wastage in descending order
top_wasted_food = food_wastage_df.sort_values(by='food_wastage', ascending=False).head(10)

# Calculate total food wastage
total_food_wastage = food_wastage_df['food_wastage'].sum()

print("Total Food Wastage:", total_food_wastage)
print("\nTop 10 Wasted Food Entree Descriptions:")
print(top_wasted_food)

Total Food Wastage: 257293

Top 10 Wasted Food Entree Descriptions:
                    entree_description  planned_entree_count  \
100                      Zatar chicken                137864   
56            Impossible Meatball Bowl                 87523   
6                       Belgium waffle                 55022   
79                      Scrambled eggs                 60331   
9                 Brioche cheeseburger                 18693   
95           Tomato and kale baked egg                  9426   
26       Classic selection cheese tray                  8937   
16               Chicken breast entree                  7821   
98   Vegetarian chili and polenta cake                  4516   
64          Mediterranean Falafel Bowl                  4054   

     consumed_entree_count  food_wastage  
100                  63307         74557  
56                   35155         52368  
6                    23037         31985  
79                   31439         28892  
9           